In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from modelAlpha128 import _netlocalWD,_netWG
#from modelAlpha256 import _netlocalWD,_netWG
import utils

In [2]:
class opt():
    def __init__(self):
        self.pngdataroot = 'dataset/pngset' # 천공영역 데이터셋 위치
        self.dataroot='dataset/animation_sao' # 트레이닝 데이터셋 위치
        self.workers=2
        self.batchSize=64 #'input batch size') 배치사이즈
        self.imageSize=128 #the height / width of the input image to network') 이미지 사이즈 (정방)
        self.nz=100
        self.ngf=128
        self.ndf=128 # center image size 중앙 천공영역 사이즈
        self.nc=4 # 이미지 채널 갯수 (RGBA)
        self.niter=10000 # iteration 횟수
        self.lr=0.0002 # 러닝레이트
        self.beta1=0.5
        self.cuda=True # 쿠다 옵션 (CPU 트레이닝 일시에는 False)
        self.ngpu=1
        #self.netG='' # 트레이닝 처음 시작할때 이 주석을 해제함
        #self.netD='' # 트레이닝 처음 시작할때 이 주석을 해제함
        self.netG='model/netG_streetview_0.pth' # 트레이닝을 이어서 할때 이 주석을 해제함
        self.netD='model/netlocalD_0.pth' # 트레이닝을 이어서 할때 이 주석을 해제함
        self.outf='.'
        self.manualSeed = 0
        self.nBottleneck=4000  # 'of dim for bottleneck of encoder') 네트워크 바틀넥 깊이
        self.overlapPred = 0 # 'overlapping edges') 청공역역 오버랩팅 사이즈
        self.nef = 128 #'of encoder filters in first conv layer') # 네트워크 필터의 갯수
        self.wtl2 = 0.998 #'0 means do not use else use with this weight')
        self.wtlD =0.001# means do not use else use with this weight')
        self.jittering=False # 천공영역 jittering 여부


In [3]:
# opt 클래스 오브젝트 생성
opt = opt()

In [4]:
# 랜덤 시드 생성, 쿠다 옵션 확인
if opt.manualSeed is None:
        opt.manualSeed = random.randint(1, 10000)
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
if opt.cuda:
    torch.cuda.manual_seed_all(opt.manualSeed)

cudnn.benchmark = True


Random Seed:  0


In [5]:
# 쿠다가 가능한데 쿠다옵션이 False 일때 알람을 띄움
if torch.cuda.is_available() and not opt.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

# 트레이닝 데이터셋과 천공영역 데이터셋의 트랜스폼 선언
transform = transforms.Compose([transforms.Resize(opt.imageSize+64),
                                transforms.RandomCrop(opt.imageSize, padding=0),
                                transforms.ToTensor()])
transform_png = transforms.Compose([transforms.Resize(opt.ndf),
                                    transforms.RandomRotation((-20,20)),
                                    transforms.RandomVerticalFlip(),
                                    transforms.RandomCrop(opt.ndf, padding=0),
                                    transforms.ToTensor()])
dataset = dset.ImageFolder(root=opt.dataroot, transform=transform)
pngdataset = dset.ImageFolder(root=opt.pngdataroot, transform=transform_png)
assert dataset
assert pngdataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize,
                                         shuffle=True, num_workers=int(opt.workers))
pngdataloader = torch.utils.data.DataLoader(pngdataset, batch_size=opt.batchSize,
                                          shuffle=True, num_workers=int(opt.workers))

print("data loading has been done")

data loading has been done


In [6]:
# 사용하지 않는 코드
ngpu = int(opt.ngpu)
ngf = int(opt.ngf)
ndf = int(opt.ndf)
nc = 3
nef = int(opt.nef)
nBottleneck = int(opt.nBottleneck)
wtl2 = float(opt.wtl2)
overlapL2Weight = 10

print("Setting options has been done")

Setting options has been done


In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


resume_epoch=0

# 네트워크와 파라메터값을 가져옴
netG = _netWG(opt)
netG.apply(weights_init)
# netG가 null path 라면 새로 생성함 (트레이닝이 처음이라면)
if opt.netG != '':
    netG.load_state_dict(torch.load(opt.netG,map_location=lambda storage, location: storage)['state_dict'])
    resume_epoch = torch.load(opt.netG)['epoch']
print(netG)

# netD가 null path 라면 새로 생성함 (트레이닝이 처음이라면)
netD = _netlocalWD(opt)
netD.apply(weights_init)
if opt.netD != '':
    netD.load_state_dict(torch.load(opt.netD,map_location=lambda storage, location: storage)['state_dict'])
    resume_epoch = torch.load(opt.netD)['epoch']

# criterion class 생성
criterion = nn.BCELoss()
criterionMSE = nn.MSELoss()

# 사용하는 텐서 만들기
input_cropped = torch.FloatTensor(opt.batchSize, 4, opt.imageSize, opt.imageSize) # 청공된 이미지가 담길 텐서
input_png = torch.FloatTensor(opt.batchSize,1, opt.ndf, opt.ndf) # 천공영역 이미지가 담길 텐서
input_pngReverse = torch.FloatTensor(opt.batchSize,4, opt.ndf, opt.ndf) # 천공영역을 반전 시킨 텐서
label = torch.FloatTensor(opt.batchSize) # label 텐서
real_label = 1
fake_label = 0

#image_margin = int((opt.imageSize - opt.ndf)/2)
image_margin = 0 # 천공영역의 마진

print(opt.batchSize)
print(opt.imageSize)

_netWG(
  (main): Sequential(
    (0): Conv2d(4, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(0.2, inplace)
    (2): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU(0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU(0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU(0.2, inplace)
    (11): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (12): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
    (13): LeakyReLU(0.2, inplace)
    (14): Conv2d(1024, 4000, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (15): BatchNorm2d(40

In [8]:
real_center = torch.FloatTensor(opt.batchSize, 4, opt.ndf, opt.ndf) #원본이미지 텐서
#real_center = torch.FloatTensor(64, 3, 64,64)

# 다음 변수들을 쿠다 옵션으로 올림
if opt.cuda:
    netD.cuda()
    netG.cuda()
    criterion.cuda()
    criterionMSE.cuda()
    input_cropped,label = input_cropped.cuda(), label.cuda()
    real_center = real_center.cuda()
    input_png = input_png.cuda()


input_cropped = Variable(input_cropped)
label = Variable(label)
input_png = Variable(input_png)
input_pngReverse = Variable(input_pngReverse)


real_center = Variable(real_center)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))

print("Setup Optimizer has been done")

Setup Optimizer has been done


In [9]:
pngmx = 93 + 77 #천공 이미지 갯수 ( pngdataroot 에 해당하는 이미지의 갯수)
for epoch in range(resume_epoch,opt.niter):      
    # 천공영역 jittering
    randwf = random.uniform(-1.0,1.0)
    randhf = random.uniform(-1.0,1.0)
    if opt.jittering:
        jitterSizeW = int(opt.imageSize/5*randwf)
        jitterSizeH = int(opt.imageSize/5*randhf)
        print("jittering : W > ",jitterSizeW," H >",jitterSizeH)
    else :
        jitterSizeW = 0
        jitterSizeH = 0

    pngdata = list(enumerate(pngdataloader, 0))[0][1][0]
    for i, data in enumerate(dataloader, 0):
        png_image = pngdata
        #print(png_image.size())
        real_cpu, _ = data
        real_cpuV =  Variable(real_cpu)
        real_center_cpu = real_cpu
        batch_size = real_cpu.size(0)
        #input_cropped.data.resize_(real_cpu.size()).copy_(real_cpu)
        #real_center.data.resize_(real_center_cpu.size()).copy_(real_center_cpu)
        
        input_cropped.data.resize_(torch.Size([batch_size, opt.nc, opt.imageSize, opt.imageSize]))
        real_center.data.resize_(torch.Size([batch_size, opt.nc, opt.ndf, opt.ndf]))
        #print("batch_size : ",batch_size , " input_cropped : ",input_cropped.data.size())
        #print("real_cetner", real_center.data[:,0].size(), " real_cetner_cpu", real_center_cpu[:,0].size())
        for j in range(0,batch_size):
            #┌----------------------------------------------------------------------------------------------┐
            # author : LJW                                                                                 
            # date 180531 
            # des : 텐서의 채널이 의미하는 것은 다음과 같음
            # tensor.data[j,i] = j 번째 이미지의 i 번째 채널 (0 : Red , 1 : Green , 2 : Blue, 3 : Alpha)
            # 단 천공이미지 텐서는 채널이 한개이기때문에 흑백으로만 이미지를 표현함.
            #└----------------------------------------------------------------------------------------------┘
            # 원본이미지 가져옴
            real_center.data[j,0] = real_center_cpu[j,0]
            real_center.data[j,1] = real_center_cpu[j,1]
            real_center.data[j,2] = real_center_cpu[j,2]
            real_center.data[j,3] = 1
            # 천공텐서에 원본이미지 넣음
            input_cropped.data[j,0] = real_cpuV.data[j,0]
            input_cropped.data[j,1] = real_cpuV.data[j,1]
            input_cropped.data[j,2] = real_cpuV.data[j,2]
            input_cropped.data[j,3] = 1
            # 천공이미지 텐서에 천공이미지 넣음
            input_png.data[j%pngmx,0] = torch.abs(png_image[j%pngmx,0] - 1)
            input_pngReverse.data[j%pngmx,0] = png_image[j%pngmx,0]
            input_pngReverse.data[j%pngmx,1] = png_image[j%pngmx,0]
            input_pngReverse.data[j%pngmx,2] = png_image[j%pngmx,0]
            input_pngReverse.data[j%pngmx,3] = png_image[j%pngmx,0]
            # 두 텐서를 Hadamard 곱을 통하여 원본이미지에 천공을 냄
            input_cropped.data[j,0,
                   int(image_margin+opt.overlapPred+jitterSizeW):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeW),
                   int(image_margin+opt.overlapPred+jitterSizeH):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeH)] = input_png.data[j%pngmx,0] * input_cropped.data[j,0,
                   int(image_margin+opt.overlapPred+jitterSizeW):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeW),
                   int(image_margin+opt.overlapPred+jitterSizeH):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeH)]
            input_cropped.data[j,1,
                   int(image_margin+opt.overlapPred+jitterSizeW):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeW),
                   int(image_margin+opt.overlapPred+jitterSizeH):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeH)] = input_png.data[j%pngmx,0] * input_cropped.data[j,1,
                   int(image_margin+opt.overlapPred+jitterSizeW):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeW),
                   int(image_margin+opt.overlapPred+jitterSizeH):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeH)]
            input_cropped.data[j,2,
                   int(image_margin+opt.overlapPred+jitterSizeW):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeW),
                   int(image_margin+opt.overlapPred+jitterSizeH):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeH)] = input_png.data[j%pngmx,0] * input_cropped.data[j,2,
                   int(image_margin+opt.overlapPred+jitterSizeW):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeW),
                   int(image_margin+opt.overlapPred+jitterSizeH):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeH)]
            input_cropped.data[j,3,
                   int(image_margin+opt.overlapPred+jitterSizeW):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeW),
                   int(image_margin+opt.overlapPred+jitterSizeH):int(image_margin+opt.ndf-opt.overlapPred+jitterSizeH)] = input_png.data[j%pngmx,0]

        ############################
        # (1) Update D network
        ###########################
        # train with real
        netD.zero_grad()
        label.data.resize_(batch_size).fill_(real_label)
        
        #print("real_center size :",real_center.size(),", label size:",label.data.size())
        output = netD(real_center)
        #print("output size:",output.size())
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.data.mean()

        # train with fake
        fake = netG(input_cropped)
        #print("fake size : ",fake.size())
        label.data.fill_(fake_label)
        #print("fake size detach :",fake.detach().size())
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()


        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.data.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG_D = criterion(output, label)
        # errG_D.backward(retain_variables=True)

        # errG_l2 = criterionMSE(fake,real_center)
        wtl2Matrix = real_center.clone()
        wtl2Matrix.data.fill_(wtl2*overlapL2Weight)
        
        '''
        # 천공영역의 결과만을 가지고 트레이닝 함
        for j in range(0,batch_size):
            wtl2Matrix.data[j,:,
                            int(opt.overlapPred):int(opt.ndf - opt.overlapPred),
                            int(opt.overlapPred):int(opt.ndf - opt.overlapPred)] = wtl2 * input_pngReverse.data
        '''

        errG_l2 = (fake-real_center).pow(2)
        errG_l2 = errG_l2 * wtl2Matrix
        errG_l2 = errG_l2.mean()

        errG = (1-wtl2) * errG_D + wtl2 * errG_l2

        errG.backward()

        D_G_z2 = output.data.mean()
        optimizerG.step()
        
        if i == len(dataloader) - 1:
            print('[%d/%d][%d/%d] errD: %.4f errG: %.4f / %.4f l_D(x): %.4f l_D(G(z)): %.4f'
                  % (epoch, opt.niter, i, len(dataloader),
                     errD, errG,errG_l2.data[0], D_x,D_G_z1, ))
        
        if epoch % 50 == 0:
            vutils.save_image(real_cpu,'result/train/real/real_samples_epoch_%03d.png' % (epoch))            
            tmpsave = torch.FloatTensor(batch_size, 3, opt.imageSize, opt.imageSize)
            tmpsave[:,0] = input_cropped.data[:,0]
            tmpsave[:,1] = input_cropped.data[:,1]
            tmpsave[:,2] = input_cropped.data[:,2]
            vutils.save_image(tmpsave,'result/train/cropped/cropped_samples_epoch_%03d.png' % (epoch))
            tmpsave[:,0] = fake.data[:,0]
            tmpsave[:,1] = fake.data[:,1]
            tmpsave[:,2] = fake.data[:,2]
            vutils.save_image(tmpsave,'result/train/recon/recon_center_samples_epoch_%03d.png' % (epoch))

    # do checkpointing
    #print('%d/%d' % (epoch, opt.niter))
    torch.save({'epoch':epoch+1,
                'state_dict':netG.state_dict()},
                'model/netG_streetview_{0}.pth'.format(epoch%2) )
    torch.save({'epoch':epoch+1,
                'state_dict':netD.state_dict()},
                'model/netlocalD_{0}.pth'.format(epoch%2))

C:\Users\ideaconcert_ai\Anaconda3\envs\py\lib\site-packages\torch\nn\functional.py:1189: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
C:\Users\ideaconcert_ai\Anaconda3\envs\py\lib\site-packages\torch\nn\functional.py:1189: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


[9265/10000][12/13] errD: 27.6310 errG: 0.0386 / 0.0387 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9266/10000][12/13] errD: 27.6310 errG: 0.0494 / 0.0495 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9267/10000][12/13] errD: 27.6310 errG: 0.0554 / 0.0555 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9268/10000][12/13] errD: 27.6310 errG: 0.0672 / 0.0673 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9269/10000][12/13] errD: 27.6310 errG: 0.0413 / 0.0414 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9270/10000][12/13] errD: 27.6310 errG: 0.0840 / 0.0842 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9271/10000][12/13] errD: 27.6310 errG: 0.0483 / 0.0484 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9272/10000][12/13] errD: 27.6310 errG: 0.0542 / 0.0544 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9273/10000][12/13] errD: 27.6310 errG: 0.0495 / 0.0496 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9274/10000][12/13] errD: 27.6310 errG: 0.0589 / 0.0590 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9275/10000][12/13] errD: 27.6310 errG: 0.0532 / 0.0533 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9276/10000][12/13] e

[9358/10000][12/13] errD: 27.6310 errG: 0.0607 / 0.0609 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9359/10000][12/13] errD: 27.6310 errG: 0.0615 / 0.0616 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9360/10000][12/13] errD: 27.6310 errG: 0.0611 / 0.0613 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9361/10000][12/13] errD: 27.6310 errG: 0.0389 / 0.0390 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9362/10000][12/13] errD: 27.6310 errG: 0.0459 / 0.0460 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9363/10000][12/13] errD: 27.6310 errG: 0.0464 / 0.0465 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9364/10000][12/13] errD: 27.6310 errG: 0.0522 / 0.0523 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9365/10000][12/13] errD: 27.6310 errG: 0.0720 / 0.0721 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9366/10000][12/13] errD: 27.6310 errG: 0.0450 / 0.0451 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9367/10000][12/13] errD: 27.6310 errG: 0.0549 / 0.0550 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9368/10000][12/13] errD: 27.6310 errG: 0.0565 / 0.0566 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9369/10000][12/13] e

[9451/10000][12/13] errD: 27.6310 errG: 0.0448 / 0.0449 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9452/10000][12/13] errD: 27.6310 errG: 0.0421 / 0.0422 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9453/10000][12/13] errD: 27.6310 errG: 0.0615 / 0.0616 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9454/10000][12/13] errD: 27.6310 errG: 0.0434 / 0.0434 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9455/10000][12/13] errD: 27.6310 errG: 0.0354 / 0.0355 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9456/10000][12/13] errD: 27.6310 errG: 0.0564 / 0.0565 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9457/10000][12/13] errD: 27.6310 errG: 0.0411 / 0.0412 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9458/10000][12/13] errD: 27.6310 errG: 0.0536 / 0.0537 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9459/10000][12/13] errD: 27.6310 errG: 0.0252 / 0.0253 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9460/10000][12/13] errD: 27.6310 errG: 0.0423 / 0.0423 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9461/10000][12/13] errD: 27.6310 errG: 0.0444 / 0.0445 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9462/10000][12/13] e

[9544/10000][12/13] errD: 27.6310 errG: 0.0640 / 0.0641 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9545/10000][12/13] errD: 27.6310 errG: 0.0557 / 0.0559 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9546/10000][12/13] errD: 27.6310 errG: 0.0426 / 0.0427 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9547/10000][12/13] errD: 27.6310 errG: 0.0657 / 0.0659 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9548/10000][12/13] errD: 27.6310 errG: 0.0594 / 0.0596 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9549/10000][12/13] errD: 27.6310 errG: 0.0323 / 0.0323 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9550/10000][12/13] errD: 27.6310 errG: 0.0732 / 0.0733 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9551/10000][12/13] errD: 27.6310 errG: 0.0447 / 0.0448 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9552/10000][12/13] errD: 27.6310 errG: 0.0423 / 0.0424 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9553/10000][12/13] errD: 27.6310 errG: 0.0512 / 0.0513 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9554/10000][12/13] errD: 27.6310 errG: 0.0507 / 0.0508 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9555/10000][12/13] e

[9637/10000][12/13] errD: 27.6310 errG: 0.0503 / 0.0504 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9638/10000][12/13] errD: 27.6310 errG: 0.0342 / 0.0343 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9639/10000][12/13] errD: 27.6310 errG: 0.0408 / 0.0409 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9640/10000][12/13] errD: 27.6310 errG: 0.0302 / 0.0302 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9641/10000][12/13] errD: 27.6310 errG: 0.0404 / 0.0405 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9642/10000][12/13] errD: 27.6310 errG: 0.0691 / 0.0692 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9643/10000][12/13] errD: 27.6310 errG: 0.0430 / 0.0430 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9644/10000][12/13] errD: 27.6310 errG: 0.0735 / 0.0737 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9645/10000][12/13] errD: 27.6310 errG: 0.0707 / 0.0709 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9646/10000][12/13] errD: 27.6310 errG: 0.0464 / 0.0465 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9647/10000][12/13] errD: 27.6310 errG: 0.0504 / 0.0505 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9648/10000][12/13] e

[9730/10000][12/13] errD: 27.6310 errG: 0.0564 / 0.0565 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9731/10000][12/13] errD: 27.6310 errG: 0.0461 / 0.0462 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9732/10000][12/13] errD: 27.6310 errG: 0.0522 / 0.0523 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9733/10000][12/13] errD: 27.6310 errG: 0.0475 / 0.0476 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9734/10000][12/13] errD: 27.6310 errG: 0.0499 / 0.0500 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9735/10000][12/13] errD: 27.6310 errG: 0.0313 / 0.0314 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9736/10000][12/13] errD: 27.6310 errG: 0.0508 / 0.0509 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9737/10000][12/13] errD: 27.6310 errG: 0.0883 / 0.0884 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9738/10000][12/13] errD: 27.6310 errG: 0.0743 / 0.0744 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9739/10000][12/13] errD: 27.6310 errG: 0.0661 / 0.0662 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9740/10000][12/13] errD: 27.6310 errG: 0.0423 / 0.0424 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9741/10000][12/13] e

[9823/10000][12/13] errD: 27.6310 errG: 0.0419 / 0.0420 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9824/10000][12/13] errD: 27.6310 errG: 0.0329 / 0.0330 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9825/10000][12/13] errD: 27.6310 errG: 0.0470 / 0.0471 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9826/10000][12/13] errD: 27.6310 errG: 0.0690 / 0.0691 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9827/10000][12/13] errD: 27.6310 errG: 0.0472 / 0.0473 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9828/10000][12/13] errD: 27.6310 errG: 0.0525 / 0.0526 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9829/10000][12/13] errD: 27.6310 errG: 0.0568 / 0.0569 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9830/10000][12/13] errD: 27.6310 errG: 0.0661 / 0.0663 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9831/10000][12/13] errD: 27.6310 errG: 0.0643 / 0.0644 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9832/10000][12/13] errD: 27.6310 errG: 0.0419 / 0.0420 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9833/10000][12/13] errD: 27.6310 errG: 0.0445 / 0.0446 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9834/10000][12/13] e

[9916/10000][12/13] errD: 27.6310 errG: 0.0457 / 0.0458 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9917/10000][12/13] errD: 27.6310 errG: 0.0641 / 0.0643 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9918/10000][12/13] errD: 27.6310 errG: 0.0474 / 0.0475 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9919/10000][12/13] errD: 27.6310 errG: 0.0436 / 0.0437 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9920/10000][12/13] errD: 27.6310 errG: 0.0872 / 0.0874 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9921/10000][12/13] errD: 27.6310 errG: 0.0353 / 0.0354 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9922/10000][12/13] errD: 27.6310 errG: 0.0361 / 0.0362 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9923/10000][12/13] errD: 27.6310 errG: 0.0395 / 0.0395 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9924/10000][12/13] errD: 27.6310 errG: 0.0665 / 0.0667 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9925/10000][12/13] errD: 27.6310 errG: 0.0424 / 0.0425 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9926/10000][12/13] errD: 27.6310 errG: 0.0407 / 0.0407 l_D(x): 1.0000 l_D(G(z)): 1.0000
[9927/10000][12/13] e